<a href="https://colab.research.google.com/github/Saksham-adh/Stocks/blob/main/STOCK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit

# Step 2: Load the data
data = pd.read_csv('/content/drive/MyDrive/NICA3yr.csv')

# Convert the index to a datetime object


# Select the required date range
start_date = '2020-01-01'
end_date = '2023-03-07'
data = data.loc[start_date:end_date]

# Step 3: Preprocess the data
data['Date'] = pd.to_datetime(data['Date'])

# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

# Drop any rows with missing values
data.dropna(inplace=True)


# Calculate Relative Strength Index (RSI)
n = 14 # RSI period
delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=n).mean()
avg_loss = loss.rolling(window=n).mean().abs()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

# Step 4: Feature engineering
data['Moving Average'] = data['Close'].rolling(window=5).mean()
data['High-Low'] = data['High'] - data['Low']
data['Difference'] = data['High'] - data['Low']
data['MA5'] = data['Close'].rolling(window=5).mean()
data['MA10'] = data['Close'].rolling(window=10).mean()

features = ['Close', 'Volume', 'Difference', 'MA5', 'MA10', 'RSI']


# Step 5: Split the data
X = data.drop('Close', axis=1)
y = data['Close']
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Step 6: Select a pretrained model
model = LinearRegression()

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Step 9 :Make predictions
future_dates = pd.date_range(start='2023-03-08', end='2023-03-15', freq='B')
future_features = data[features].loc[future_dates]
future_predictions = model.predict(future_features)

# Create a DataFrame of future predictions
future_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted Close Price': future_predictions
})

# Set the "Date" column as the index
future_df.set_index('Date', inplace=True)

# Step 10: Visualize the results
plt.plot(future_data.index, future_y, label='Predicted')
plt.plot(future_data.index, future_data['Close'], label='Actual')
plt.legend()
plt.show()
